In [46]:
import numpy as np
import pandas as pd

In [47]:
df_accidentes = pd.read_excel('df_accidentes_analisis.xlsx')

# KPI 1: Tasa de fatalidad tripulantes

La tasa de fatalidad de la tripulación como el número total de tripulantes fallecidos en los accidentes 
registrados en la década a considerar, dividido en la cantidad total de accidentes aéreos ocurridos en 
este período de tiempo. Su fórmula es:
(Suma total de fallecidos en el período de tiempo / Suma total de accidentes en el período de tiempo).

**Objetivo**
- Evaluar la disminución de un 10% la tasa de fatalidad de la tripulación en los últimos 10 años, en comparación con la década anterior 

In [48]:
df_accidentes['fecha'] = pd.to_datetime(df_accidentes['fecha'], format='%B %d, %Y')
df_accidentes['año'] = df_accidentes['fecha'].dt.year

In [49]:
df_tripulantes_fallecidos = df_accidentes[df_accidentes['crew_fatalities'] > 0]
df_tripulantes_fallecidos.head(3)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Pais,año
2,2,1912-07-12,630.0,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,New Jersey,1912
3,3,1913-08-06,NaN,"Victoria, British Columbia, Canada",Private,None,None,Curtiss seaplane,None,None,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...,Canada,1913
6,6,1915-03-05,100.0,"Tienen, Belgium",Military - German Navy,None,None,Zeppelin L-8 (airship),None,None,41.0,0.0,41.0,17.0,0.0,17.0,0.0,Crashed into trees while attempting to land af...,Belgium,1915


In [50]:
df_tripulantes_fallecidos = df_tripulantes_fallecidos.groupby('año').agg({'crew_aboard': 'sum', 'Unnamed: 0': 'count'})
df_tripulantes_fallecidos = df_tripulantes_fallecidos.rename(columns={'crew_aboard': 'suma_fallecidos', 'Unnamed: 0': 'cantidad_Accidentes'})
df_tripulantes_fallecidos = df_tripulantes_fallecidos.reset_index()
df_tripulantes_fallecidos

,año,suma_fallecidos,cantidad_Accidentes
0,1912,5.0,1
1,1913,1.0,1
2,1915,41.0,1
3,1916,19.0,1
4,1918,1.0,1
...,...,...,...
103,2017,43.0,13
104,2018,86.0,14
105,2019,52.0,13
106,2020,45.0,7


In [51]:
bins = range(df_tripulantes_fallecidos['año'].min(), df_tripulantes_fallecidos['año'].max() + 11, 10)
bins

range(1912, 2032, 10)

In [52]:
df_tripulantes_fallecidos['periodo'] = pd.cut(df_tripulantes_fallecidos['año'], bins=bins, right=False)
df_tripulantes_fallecidos

,año,suma_fallecidos,cantidad_Accidentes,periodo
0,1912,5.0,1,"[1912, 1922)"
1,1913,1.0,1,"[1912, 1922)"
2,1915,41.0,1,"[1912, 1922)"
3,1916,19.0,1,"[1912, 1922)"
4,1918,1.0,1,"[1912, 1922)"
...,...,...,...,...
103,2017,43.0,13,"[2012, 2022)"
104,2018,86.0,14,"[2012, 2022)"
105,2019,52.0,13,"[2012, 2022)"
106,2020,45.0,7,"[2012, 2022)"


In [53]:
resultado = df_tripulantes_fallecidos.groupby('periodo').agg({'suma_fallecidos': 'sum', 'cantidad_Accidentes': 'sum'})
resultado = resultado.reset_index()
resultado

,periodo,suma_fallecidos,cantidad_Accidentes
0,"[1912, 1922)",155.0,39
1,"[1922, 1932)",364.0,187
2,"[1932, 1942)",1021.0,306
3,"[1942, 1952)",2611.0,597
4,"[1952, 1962)",2809.0,564
5,"[1962, 1972)",2967.0,592
6,"[1972, 1982)",2577.0,504
7,"[1982, 1992)",2205.0,478
8,"[1992, 2002)",2495.0,539
9,"[2002, 2012)",1660.0,409


In [54]:
resultado['tasa'] = resultado['suma_fallecidos'] / resultado['cantidad_Accidentes']
resultado

,periodo,suma_fallecidos,cantidad_Accidentes,tasa
0,"[1912, 1922)",155.0,39,3.974359
1,"[1922, 1932)",364.0,187,1.946524
2,"[1932, 1942)",1021.0,306,3.336601
3,"[1942, 1952)",2611.0,597,4.373534
4,"[1952, 1962)",2809.0,564,4.980496
5,"[1962, 1972)",2967.0,592,5.011824
6,"[1972, 1982)",2577.0,504,5.113095
7,"[1982, 1992)",2205.0,478,4.612971
8,"[1992, 2002)",2495.0,539,4.628942
9,"[2002, 2012)",1660.0,409,4.058680


In [55]:
resultado['disminucion'] = resultado['tasa'] - resultado['tasa'].pct_change() * 100
resultado

,periodo,suma_fallecidos,cantidad_Accidentes,tasa,disminucion
0,"[1912, 1922)",155.0,39,3.974359,NaN
1,"[1922, 1932)",364.0,187,1.946524,52.969467
2,"[1932, 1942)",1021.0,306,3.336601,-68.076708
3,"[1942, 1952)",2611.0,597,4.373534,-26.703988
4,"[1952, 1962)",2809.0,564,4.980496,-8.897573
5,"[1962, 1972)",2967.0,592,5.011824,4.382813
6,"[1972, 1982)",2577.0,504,5.113095,3.092456
7,"[1982, 1992)",2205.0,478,4.612971,14.394219
8,"[1992, 2002)",2495.0,539,4.628942,4.282706
9,"[2002, 2012)",1660.0,409,4.058680,16.378184


In [56]:
resultado.to_excel('Tripulacion.xlsx', index = False)

# KPI 2: Tasa fallecidos

La tasa de fallecidos de la tripulación como el número total de fallecidos en los accidentes 
registrados en la década a considerar, dividido en la cantidad total de accidentes aéreos ocurridos en 
este período de tiempo. Su fórmula es:
(Suma total de fallecidos en el período de tiempo / Suma total de accidentes en el período de tiempo).

**Objetivo**
- Evaluar la disminución de un 20% la tasa de fallecidos en los últimos 10 años, en comparación con la década anterior 

In [60]:
df_fallecidos = df_accidentes[df_accidentes['cantidad de fallecidos'] > 0]
df_fallecidos.head(3)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Pais,año
0,0,1908-09-17,1718.0,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,Wright Flyer III,None,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly...",United States,1908
1,1,1909-09-07,NaN,"Juvisy-sur-Orge, France",None,None,Air show,Wright Byplane,SC1,None,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...,France,1909
2,2,1912-07-12,630.0,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,New Jersey,1912


In [62]:
df_fallecidos = df_fallecidos.groupby('año').agg({'cantidad de fallecidos': 'sum', 'Unnamed: 0': 'count'})
df_fallecidos = df_fallecidos.rename(columns={'cantidad de fallecidos': 'suma_fallecidos', 'Unnamed: 0': 'cantidad_Accidentes'})
df_fallecidos = df_fallecidos.reset_index()
df_fallecidos

,año,suma_fallecidos,cantidad_Accidentes
0,1908,1.0,1
1,1909,1.0,1
2,1912,5.0,1
3,1913,43.0,3
4,1915,36.0,2
...,...,...,...
106,2017,214.0,15
107,2018,874.0,19
108,2019,305.0,13
109,2020,355.0,8


In [63]:
bins = range(df_fallecidos['año'].min(), df_fallecidos['año'].max() + 11, 10)
bins

range(1908, 2032, 10)

In [64]:
df_fallecidos['periodo'] = pd.cut(df_fallecidos['año'], bins=bins, right=False)
df_fallecidos

,año,suma_fallecidos,cantidad_Accidentes,periodo
0,1908,1.0,1,"[1908, 1918)"
1,1909,1.0,1,"[1908, 1918)"
2,1912,5.0,1,"[1908, 1918)"
3,1913,43.0,3,"[1908, 1918)"
4,1915,36.0,2,"[1908, 1918)"
...,...,...,...,...
106,2017,214.0,15,"[2008, 2018)"
107,2018,874.0,19,"[2018, 2028)"
108,2019,305.0,13,"[2018, 2028)"
109,2020,355.0,8,"[2018, 2028)"


In [65]:
resultado2 = df_fallecidos.groupby('periodo').agg({'suma_fallecidos': 'sum', 'cantidad_Accidentes': 'sum'})
resultado2 = resultado2.reset_index()
resultado2

,periodo,suma_fallecidos,cantidad_Accidentes
0,"[1908, 1918)",332.0,20
1,"[1918, 1928)",486.0,116
2,"[1928, 1938)",1680.0,339
3,"[1938, 1948)",7079.0,511
4,"[1948, 1958)",11748.0,655
5,"[1958, 1968)",15587.0,637
6,"[1968, 1978)",20278.0,631
7,"[1978, 1988)",15987.0,506
8,"[1988, 1998)",17768.0,648
9,"[1998, 2008)",11749.0,519


In [66]:
resultado2['tasa'] = resultado2['suma_fallecidos'] / resultado2['cantidad_Accidentes']
resultado2

,periodo,suma_fallecidos,cantidad_Accidentes,tasa
0,"[1908, 1918)",332.0,20,16.600000
1,"[1918, 1928)",486.0,116,4.189655
2,"[1928, 1938)",1680.0,339,4.955752
3,"[1938, 1948)",7079.0,511,13.853229
4,"[1948, 1958)",11748.0,655,17.935878
5,"[1958, 1968)",15587.0,637,24.469388
6,"[1968, 1978)",20278.0,631,32.136292
7,"[1978, 1988)",15987.0,506,31.594862
8,"[1988, 1998)",17768.0,648,27.419753
9,"[1998, 2008)",11749.0,519,22.637765


In [67]:
resultado2['disminucion'] = resultado2['tasa'] - resultado2['tasa'].pct_change() * 100
resultado2

,periodo,suma_fallecidos,cantidad_Accidentes,tasa,disminucion
0,"[1908, 1918)",332.0,20,16.600000,NaN
1,"[1918, 1928)",486.0,116,4.189655,78.950769
2,"[1928, 1938)",1680.0,339,4.955752,-13.329692
3,"[1938, 1948)",7079.0,511,13.853229,-165.685141
4,"[1948, 1958)",11748.0,655,17.935878,-11.534861
5,"[1958, 1968)",15587.0,637,24.469388,-11.957655
6,"[1968, 1978)",20278.0,631,32.136292,0.803657
7,"[1978, 1988)",15987.0,506,31.594862,33.279655
8,"[1988, 1998)",17768.0,648,27.419753,40.634271
9,"[1998, 2008)",11749.0,519,22.637765,40.077704


In [68]:
resultado2.to_excel('Total.xlsx', index = False)